In [114]:
from scipy.stats import mannwhitneyu
import pandas as pd
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import kruskal


In [106]:

file_path = '/content/drive/My Drive/Colab Notebooks/lol.xlsx'
df = pd.read_excel(file_path, engine='openpyxl')
df


,Unnamed: 0,Фамилия Имя,Возраст игрока,Национальность,Позиция на поле,Лига,Клуб,Стоимость,Турнир,Минут на поле в розыгрыше,...,Прогрессивный пас,Прогресивные пасы не дошедшие до получателя,Коэффициент голов за минуту,Коэффициент голевых передач за минуту,Коэффициент голов + голевых передач за минуту,Коэффициент прогрессивных переносов за минуту,Коэффициент прогрессивных пасов за минуту,Коэффициент неудачных прогрессивных пасов за минуту,Успешные прогрессивные пасы,Коэффициент успешных прогрессивных пасов
0,0,Kylian Mbappé,25,Франция,Центральный нап.,Лига 1,Пари Сен-Жермен,180,Лига Чемпионов,900,...,56,125,0.008889,0.000000,0.008889,0.072222,0.062222,0.138889,-69,-1.232143
1,1,Erling Haaland,23,Норвегия,Центральный нап.,АПЛ,Манчестер Сити,180,Лига Чемпионов,776,...,11,43,0.007732,0.001289,0.009021,0.009021,0.014175,0.055412,-32,-2.909091
2,2,Jude Bellingham,20,Англия,Атак. полузащитник,Ла лига,Реал Мадрид,180,Лига Чемпионов,742,...,56,56,0.005391,0.005391,0.010782,0.039084,0.075472,0.075472,0,0.000000
3,3,Bukayo Saka,22,Англия,Правый Вингер,АПЛ,Арсенал,130,Лига Чемпионов,720,...,22,89,0.005556,0.005556,0.011111,0.056944,0.030556,0.123611,-67,-3.045455
4,4,Phil Foden,23,Англия,Правый Вингер,АПЛ,Манчестер Сити,130,Лига Чемпионов,681,...,53,89,0.007342,0.005874,0.013216,0.033774,0.077827,0.130690,-36,-0.679245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,201,Fredrik Aursnes,28,Норвегия,Опорный полузащитник,annother leage,Бенфика,20,Лига Чемпионов,540,...,24,37,0.000000,0.001852,0.001852,0.018519,0.044444,0.068519,-13,-0.541667
202,202,Álvaro Morata,31,Испания,Центральный нап.,Ла лига,Атлетико Мадрид,20,Лига Чемпионов,665,...,6,39,0.007519,0.001504,0.009023,0.018045,0.009023,0.058647,-33,-5.500000
203,203,Oscar Gloukh,20,Израиль,Атак. полузащитник,annother leage,Ред Булл Зальцбург,20,Лига Чемпионов,476,...,25,26,0.004202,0.002101,0.006303,0.033613,0.052521,0.054622,-1,-0.040000
204,204,Miguel Almirón,30,Парагвай,Правый Вингер,АПЛ,Ньюкасл Юнайтед,20,Лига Чемпионов,413,...,23,30,0.002421,0.002421,0.004843,0.043584,0.055690,0.072639,-7,-0.304348


## У нас есть несколько гипотез:первая из них существует в футбольном мире и я бы хотел ее проверить.Существует устойчивый миф(или не миф,сейчас узнаем),что **АПЛ лига ,в кототрой команлы больше всего любят атаковать**,а в итальянской Серии А ,наоборот не любят позиционные атаки и заточены на обороне

In [107]:
df_filtered=df[['Фамилия Имя', 'Лига', 'Коэффициент голов + голевых передач за минуту','Коэффициент прогрессивных переносов за минуту', 'Коэффициент прогрессивных пасов за минуту']]
#Создадим сортированный Дата фрэйм для использования только нужных атакуюищх метрик
#Создадим дополнительный признак по сумме атакуюищх дейсвий
df_filtered['Сумма атакующих действий'] = (df_filtered['Коэффициент голов + голевых передач за минуту'] +df_filtered['Коэффициент прогрессивных переносов за минуту'] +df_filtered['Коэффициент прогрессивных пасов за минуту'])
df_apl = df_filtered[df_filtered['Лига'] == 'АПЛ']
df_serie_a = df_filtered[df_filtered['Лига'] == 'Серия А']
apl_attacking_actions = df_apl['Сумма атакующих действий']
serie_a_attacking_actions = df_serie_a['Сумма атакующих действий']
t_test = ttest_ind(apl_attacking_actions, serie_a_attacking_actions, equal_var=False)
u_test = mannwhitneyu(apl_attacking_actions, serie_a_attacking_actions)
#Расчеты по дефолту делаются для уровня значимости =0.05
print(u_test)
print(t_test)




MannwhitneyuResult(statistic=982.0, pvalue=0.026243991389969433)
TtestResult(statistic=2.912455536022612, pvalue=0.0049465049620424206, df=63.374999147409234)


<ipython-input-107-d0e9571eeb45>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Сумма атакующих действий'] = (df_filtered['Коэффициент голов + голевых передач за минуту'] +df_filtered['Коэффициент прогрессивных переносов за минуту'] +df_filtered['Коэффициент прогрессивных пасов за минуту'])


Мы проводили тест на равенство атакуюищх действий , но по результатам тестов,можно сказать , ***что АПЛ  более атакующая лига***.Статистика=982,p-value-0,02,то есть значение при котором отвергается нулевая гипотеза о равенстве(отвергается в пользу АПЛ,Статистика=982).T-test сделал больше для красоты и подтверждения,потому что он вроде не так хорошо работает с выборками с разными дисперсиями и данные вроде не обладают нормальным распределением,что так же желательно для t-тестов.

### **ВАЖНО,я щас буду пробовать бустрап,мы его не проходили,так что я буду пробовать с нуля по гайдам в инете,так что если че не суди строго**

In [108]:
def mean_d(k, f):
    return np.mean(k) - np.mean(f)
#это вроде не машинка,и я не уверен ,как тут будет работать MSE,MAE и прочие всякие штуки,так что я ручками написал просто функцию
apl_attacking_actions_v = df_apl['Сумма атакующих действий'].values
serie_a_attacking_actions_v = df_serie_a['Сумма атакующих действий'].values

# Объединение массивов
combined_actions = np.append(apl_attacking_actions_v, serie_a_attacking_actions_v)
bootstrap_diffs = np.zeros(4321)
#вроде как тут именно масив сохраняется и с ним поудобнее работать
for i in range(4321):
    sample_apl = np.random.choice(apl_attacking_actions, size=len(apl_attacking_actions), replace=True)#бустрап вроде делают с возвратом,по крайнеей мере пишут так,хотя навернео из-за неопытности мне кажется это странным
    sample_serie_a = np.random.choice(serie_a_attacking_actions, size=len(serie_a_attacking_actions), replace=True)
    bootstrap_diffs[i] = mean_d(sample_apl, sample_serie_a)
lower = np.percentile(bootstrap_diffs, 2.5)
upper = np.percentile(bootstrap_diffs, 97.5)
print(lower,upper)




0.008626341035364166 0.04084503992502227


ну получается ,что с 95 процентной вероястностью,можно сказать,что(вроде так работает) в пересчете на игровое время у игроков АПЛ больше голевых действий.То есть разница между игроками АПЛ и игроками Серия А с 95 проц вероястностью лежитв  этом интервале [0.008638831996706176 0.04137916484834663]

Следующая гипотеза:провреим является ли Манчестер сити самой атакующей командой в ЛЧ,в среднем по данным ,если просто взять сумму атакующих действий ,то графики показывают,что это так,проверим на тестах


In [109]:
df['Сумма атакующих действий'] = (df['Коэффициент голов + голевых передач за минуту'] +df['Коэффициент прогрессивных переносов за минуту'] +df['Коэффициент прогрессивных пасов за минуту'])
df_manchester_city = df[df['Клуб'] == 'Манчестер Сити']
df_other_teams = df[df['Клуб'] != 'Манчестер Сити']
manchester_city_attacking_actions = df_manchester_city['Сумма атакующих действий'].dropna()
other_teams_attacking_actions = df_other_teams['Сумма атакующих действий'].dropna()
t_test_result_MC = ttest_ind(manchester_city_attacking_actions, other_teams_attacking_actions, equal_var=False)
u_test_MC = mannwhitneyu(manchester_city_attacking_actions, other_teams_attacking_actions)
print(t_test_result_MC)
print(u_test_MC)

TtestResult(statistic=3.6904970264614945, pvalue=0.0012623739697923248, df=22.228872397739746)
MannwhitneyuResult(statistic=2929.0, pvalue=0.0001395490019633605)


## Опять подтверждающаяся гипотеза,pvalue сильно меньше на обоих тестах ,чем уровень 0.05 => отвергаем теорию о том,что манчестер сити равен остальным командам по атакующим действиям и принимаем гипотезу о том,что ***Манчестер сити в среднем самая атакующая команды ЛЧ***

In [110]:
def mean_i_love_footbal_lol(d1, d2):
    return np.mean(d1) - np.mean(d2)
bootstrap_diffs = np.zeros(5050)
for i in range(5050):
    sample_mc = np.random.choice(manchester_city_attacking_actions, size=len(manchester_city_attacking_actions), replace=True)
    sample_other = np.random.choice(other_teams_attacking_actions, size=len(other_teams_attacking_actions), replace=True)
    bootstrap_diffs[i] = mean_i_love_footbal_lol(sample_mc, sample_other)

low = np.percentile(bootstrap_diffs, 2.5)
up = np.percentile(bootstrap_diffs, 97.5)
print(low)
print(up)


0.021605675023485434
0.06458812102544616


Ну с бустрапом тоже вроде норм все , и гипотеза о том,что МС в среднем самая атакующая команда в ЛЧ подтверждается

Тут я хочу проверить гипотезу о том,что с точки зрения действий ведущих к голу,но без учета непосредственно реализации,равные команды.Сравниваю именно эти команды.Потому что одна команда-победитель ЛЧ 2023(МС),а другая главный претендент на победу в ЛЧ 2024(Реализацией в футболе называют условие,при котором действие вело к голу,но гол не был забит)


In [111]:
df['Прогрессивные действия'] = df['Коэффициент прогрессивных переносов за минуту'] + df['Коэффициент прогрессивных пасов за минуту']

# Filter data for Real Madrid, Manchester City and other teams
df_real_madrid = df[df['Клуб'] == 'Реал Мадрид']
df_manchester_city = df[df['Клуб'] == 'Манчестер Сити']

# Extract progressive actions sums for both groups
real_madrid_progressive_actions = df_real_madrid['Прогрессивные действия'].dropna()
manchester_city_progressive_actions = df_manchester_city['Прогрессивные действия'].dropna()

# Mann-Whitney U test
u_test_MC_RM = mannwhitneyu(manchester_city_progressive_actions, real_madrid_progressive_actions)
print(u_test_MC_RM)

MannwhitneyuResult(statistic=174.0, pvalue=0.07512082297763525)


In [112]:
def mean_d1(da1, da2):
    return np.mean(da1) - np.mean(da2)
bootstrap_diffs = np.zeros(3223)
for i in range(3223):
    sample_mc = np.random.choice(manchester_city_progressive_actions, size=len(manchester_city_progressive_actions), replace=True)
    sample_rm = np.random.choice(real_madrid_progressive_actions, size=len(real_madrid_progressive_actions), replace=True)
    bootstrap_diffs[i] = mean_d1(sample_mc, sample_rm)

lower1 = np.percentile(bootstrap_diffs, 2.5)
upper1 = np.percentile(bootstrap_diffs, 97.5)
print(lower1,upper1)



0.0036949653941625513 0.05543128220878538


Тут уже есть некоторые разногласия,НО возьмем точку зрения U теста , и скажем,что ,если говорить только лишь о действиях ведущих к голу,команды равны

Проверим ,что игроки трех основных сборных претендентов на победы на Евро2024 (Англия,Испания,Франция)**имеют равные атакующие способоности**

In [115]:
df['Сумма атакующих действий'] = (
    df['Коэффициент голов + голевых передач за минуту'] +
    df['Коэффициент прогрессивных переносов за минуту'] +
    df['Коэффициент прогрессивных пасов за минуту']
)
df_spanish = df[df['Национальность'] == 'Испания']
df_english = df[df['Национальность'] == 'Англия']
df_french = df[df['Национальность'] == 'Франция']

# Extract the sum of attacking actions for each group
spanish_attacking_actions = df_spanish['Сумма атакующих действий'].dropna()
english_attacking_actions = df_english['Сумма атакующих действий'].dropna()
french_attacking_actions = df_french['Сумма атакующих действий'].dropna()
kruskal_test_result = kruskal(spanish_attacking_actions, english_attacking_actions, french_attacking_actions)
print(, kruskal_test_result)

Kruskal-Wallis H-test result: KruskalResult(statistic=1.935218741006295, pvalue=0.37999036931029384)


Тут чуть помог Chat Gpt по запросу какой делать тест для сравнения трех выборок,он посоветовал вот этот тест ,и сказал что это специализированные тест.В целом тест показал,что гипотеза не отвергается,и что скорее всего в плане атакующего потенциала и с точки зрения индивидуальной игры,сборные равны